In [60]:
###installing dependencies
import pandas as pd
import numpy as np
import os
import glob
import yfinance as yf
import datetime
import re
import emoji
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob


In [35]:
os.chdir("/Users/pranavtavildar/Desktop/Senior-Thesis/data/monthlydata")  
#creating big csv
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
#combine all files in the list
#all_filenames.remove('CVX-Tweets-12-2022-310.csv')


combined_csv = pd.DataFrame()
i = 0
for f in all_filenames:
    try:
        df = pd.read_csv(f, encoding='utf-8-sig',lineterminator='\n')
        # df.info()
        combined_csv = pd.concat([combined_csv, df], ignore_index=True, verify_integrity=True)
        i += 1
    except Exception as e:
        print(i, "files were read and combined properly")
        # df.info()
        print(df.describe())
        print(f"Error reading file {f}: {str(e)}")
        

# #export to csv
combined_csv.to_csv( "../CVXTweets.csv", index=False, encoding='utf-8-sig')

In [45]:
os.chdir("/Users/pranavtavildar/Desktop/Senior-Thesis/data")

# Retrieve the data from Yahoo Finance
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2022, 12, 31)

cvx_data = yf.download('CVX', start=start_date, end=end_date)

cvx_data.to_csv('CVX_stock_data.csv')



[*********************100%***********************]  1 of 1 completed


In [48]:
# Loading stock data 
stock_data = pd.read_csv("CVX_stock_data.csv")
stock_data["Date"] = pd.to_datetime(stock_data["Date"])
stock_data = stock_data.set_index("Date")

# Loading tweets
tweets = pd.read_csv("CVXTweets.csv",encoding='utf-8-sig',lineterminator='\n')
tweets["Date"] = pd.to_datetime(tweets["Date"])

# Filtering tweets
filtered_tweets = tweets.loc[tweets["Date"].dt.date.isin(stock_data.index.date)]

# Save the filtered tweets to a new CSV file
filtered_tweets.to_csv("CVXTweets_filtered.csv", index=False)


In [62]:
###Preprocessing Tweets
def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    r = tweet.lower()
    r = re.sub("'", "", r) # This is to avoid removing contractions in english
    r = re.sub("@[A-Za-z0-9_]+","", r)
    r = re.sub("#[A-Za-z0-9_]+","", r)
    r = re.sub(r'http\S+', '', r)
    r = re.sub('[()!?]', ' ', r)
    r = re.sub('\[.*?\]',' ', r)
    r = re.sub("[^a-z0-9]"," ", r)
    r = r.split()
    stopwords = ["for", "on", "an", "a", "of", "and", "in", "the", "to", "from"]
    r = [w for w in r if not w in stopwords]
    r = " ".join(word for word in r)
    return r

# Apply the clean_tweet function 
filtered_tweets['cleaned_text'] = filtered_tweets['Text'].apply(clean_tweet)

filtered_tweets.to_csv('CVXTweets_cleaned.csv', index=False)

unfiltered_tweets = pd.read_csv("CVXTweets_filtered.csv",encoding='utf-8-sig',lineterminator='\n')

print(unfiltered_tweets.equals(filtered_tweets))
unfiltered_tweets.head()

/var/folders/kz/950pxqcd0zn5rw95_9btv5p00000gn/T/ipykernel_51375/2638477000.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if type(tweet) == np.float:
/var/folders/kz/950pxqcd0zn5rw95_9btv5p00000gn/T/ipykernel_51375/2638477000.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_tweets['cleaned_text'] = filtered_tweets['Text'].apply(clean_tweet)


False


,Date,Tweet ID,Text,Likes
0,2021-02-01 23:55:56+00:00,1356390859781132288,@JeremyPryer @TheSarcasmFiend @God_Complex_013...,1
1,2021-02-01 23:40:05+00:00,1356386873757868032,$BLSP Set to run this week. Biden environmenta...,6
2,2021-02-01 23:38:23+00:00,1356386446127427584,Reports this weekend that $XOM and $CVX discus...,7
3,2021-02-01 23:35:32+00:00,1356385727173525511,Eyecatching stuff: $XOM/$CVX merger rumor gets...,182
4,2021-02-01 23:14:56+00:00,1356380541906419712,@firstaidspray @TheSarcasmFiend @God_Complex_0...,0


In [69]:
###Sentiment Analysis 

filtered_tweets['sentiment'] = filtered_tweets['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.polarity)

filtered_tweets.to_csv('CVXTweets_sentiments.csv', index=False)
filtered_tweets.head()


/var/folders/kz/950pxqcd0zn5rw95_9btv5p00000gn/T/ipykernel_51375/1803340424.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_tweets['sentiment'] = filtered_tweets['cleaned_text'].apply(lambda x: TextBlob(x).sentiment.polarity)


,Date,Tweet ID,Text,Likes,cleaned_text,sentiment
0,2021-02-01 23:55:56+00:00,1356390859781132288,@JeremyPryer @TheSarcasmFiend @God_Complex_013...,1,thanks context but fact that cvx isnt switch t...,0.050000
1,2021-02-01 23:40:05+00:00,1356386873757868032,$BLSP Set to run this week. Biden environmenta...,6,blsp set run this week biden environmental pla...,-0.200000
2,2021-02-01 23:38:23+00:00,1356386446127427584,Reports this weekend that $XOM and $CVX discus...,7,reports this weekend that xom cvx discussed po...,0.100000
3,2021-02-01 23:35:32+00:00,1356385727173525511,Eyecatching stuff: $XOM/$CVX merger rumor gets...,182,eyecatching stuff xom cvx merger rumor gets no...,-0.263889
4,2021-02-01 23:14:56+00:00,1356380541906419712,@firstaidspray @TheSarcasmFiend @God_Complex_0...,0,these links direct specific sites that were se...,0.063333


In [75]:
# group the data by Date and calculate the mean sentiment
filtered_tweets['Date'] = pd.to_datetime(filtered_tweets['Date']).dt.date
avg_sentiment = filtered_tweets.groupby(['Date'])['sentiment'].mean()
filtered_tweets.to_csv('daily_sentiments.csv', index=False)
avg_sentiment.head()

/var/folders/kz/950pxqcd0zn5rw95_9btv5p00000gn/T/ipykernel_51375/2124092546.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_tweets['Date'] = pd.to_datetime(filtered_tweets['Date']).dt.date


Date
2021-01-04    0.132750
2021-01-05    0.123115
2021-01-06    0.073396
2021-01-07    0.032773
2021-01-08    0.057576
Name: sentiment, dtype: float64

In [76]:
# calculate the weighted sentiment by multiplying the sentiment by Likes
filtered_tweets['weighted_sentiment'] = filtered_tweets['sentiment'] * filtered_tweets['Likes']

# group the data by Date and calculate the weighted mean sentiment
weighted_avg_sentiment = filtered_tweets.groupby(['Date'])['weighted_sentiment', 'Likes'].sum()
weighted_avg_sentiment['weighted_sentiment'] /= weighted_avg_sentiment['Likes']

weighted_avg_sentiment.to_csv('weighted_daily_sentiments.csv', index=False)
weighted_avg_sentiment['weighted_sentiment'].head()


/var/folders/kz/950pxqcd0zn5rw95_9btv5p00000gn/T/ipykernel_51375/402456260.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_tweets['weighted_sentiment'] = filtered_tweets['sentiment'] * filtered_tweets['Likes']
/var/folders/kz/950pxqcd0zn5rw95_9btv5p00000gn/T/ipykernel_51375/402456260.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  weighted_avg_sentiment = filtered_tweets.groupby(['Date'])['weighted_sentiment', 'Likes'].sum()


Date
2021-01-04    0.250000
2021-01-05    0.399708
2021-01-06    0.405114
2021-01-07         NaN
2021-01-08    0.000000
Name: weighted_sentiment, dtype: float64